# Problem 1: Exponential integrals

In this problem, you will write code to compute the [exponential integral](https://en.wikipedia.org/wiki/Exponential_integral) function
$$
E_1(z) = \int_z^\infty \frac{e^{-t}}{t} dt
$$
For simplicity, you only need to compute it for $\Re z \ge 0$, in which case the definition can be rewritten
$$
E_1(z) = \int_0^1 \frac{e^{-z/u}}{u} du
$$
A simplistic implementation of this function is just to directly perform the integral numerically:

In [6]:
function E₁{T<:Real}(z::Union{T,Complex{T}})
    real(z) < 0 && error("real(z) < 0 not implemented")
    return quadgk(u -> exp(-z/u)/u, 0, 1, reltol=eps(T)*10)[1]
end

E₁ (generic function with 1 method)

In [2]:
E₁(1.2)

0.1584084368514626

To check this, a useful resource is [WolframAlpha](https://www.wolframalpha.com), which can compute $E_1(z)$ by `ExpIntegralE[1,z]`.  For $E_1(1.2)$, [we get](https://www.wolframalpha.com/input/?i=ExpIntegralE%5B1,1.2%5D) `0.158408436851462561424955970710861738534157976840578963378...`.  Compared to the above result, this corresponds to a relative error of:

In [9]:
# compute the relative error of approx compared to exact
relerr(approx,exact) = norm(approx - exact) / norm(exact)

relerr(E₁(1.2), 0.158408436851462561424955970710861738534157976840578963378)

3.5043052210224057e-16

Hooray, we are getting the right result to nearly machine precision!  We can even do complex values this way:

In [10]:
relerr(E₁(1.2 + 3.4im),
       -0.0196798781439709839467398951111963946354437628483798953 +
        0.0708764302707789307217865597073426047954413415009995511im)

1.886647948886305e-16

Yup, it's working for complex $z$ too!

However, this approach to computing $E_1$ is *insanely slow* compared to highly optimized implementations.   **Your task is to write**
```jl
E₁(z::Union{Float64,Complex{Float64}}) = ...?...
```
and make it much faster than the generic version above, but only for double precision (`Float64` real or complex numbers).  You should have a **relative error** (as computed by `relerr`, above) of $< 10^{-13}$ for all $z$.

Some techniques you can use are polynomial expansions (e.g. Taylor series), continued-fraction expansions, other rational approximants, along with recurrence relations and other identities.  You will find [this page listing many identities and expansions](http://functions.wolfram.com/GammaBetaErf/ExpIntegralE/) to be helpful.

# Problem 2: Parallel mapreduce

Your task is to speed up the function `myreduce(⊕, n)` that computes
$$
    1 \oplus 2 \oplus 3 \oplus \cdots \oplus n
$$
for $n \ge 0$, where $\oplus$ is a caller-supplied **commutative binary function** `⊕(x,y)`.  (You are allowed to assume commutativity to make the parallelization easier.)  A simple serial implementation is:

In [13]:
myreduce(⊕, n) = reduce(⊕, 1:n)

myreduce (generic function with 1 method)

In [14]:
myreduce(+, 1000) == 500500

true

The goal is to use parallelism.  You function will run on a parallel computer where `addprocs` *has already been called* (you are not allowed to call it), and you can use Julia's [parallel-computing constructs](http://docs.julialang.org/en/stable/manual/parallel-computing/) to parallelize the `myreduce` function.

You are allowed to [DistributedArrays](https://github.com/JuliaParallel/DistributedArrays.jl) or any other Julia package you want, if it is helpful.

**There is a catch**, however: the `⊕(x,y)` function may be *much slower* for some `x,y` than for others, and as a result it will not be efficient to simply divide the numbers `1:n` up equally between the workers.

For example, here is a `⊕` function that is much slower for smaller `x,y` than for larger `x,y`:

In [15]:
function weirdplus(x,y)
    sleep(1e-3 / (x+y)) # wait for some number of seconds
    return x + y
end
myreduce(weirdplus, 1000)

500500

You do not know in advance which arguments will be the expensive ones.  Can you still exploit parallelism?